### Spectrogram based Transfer Learning

#### Data cleanup

In [1]:
import matplotlib.pyplot as plt
import os
import subprocess

def run_preprocess(root, length, split):
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            subprocess.call(["./preprocess", os.path.join(subdir, directory), length, split])
        break

In [2]:
def load_features(root, split):
    spect_data = []
    spect_label = []
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            file_path = os.path.join(subdir, directory, "split", split, "spect")
            for filename in os.listdir(file_path):
                x = plt.imread(os.path.join(file_path, filename))
                spect_data.append(x)
                spect_label.append(directory)
        break
    return spect_data, spect_label

In [10]:
import shutil

def cleanup_split(root):
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            shutil.rmtree(os.path.join(subdir, directory, "split"), ignore_errors = True)
        break

In [12]:
cleanup_split('audio-train')
run_preprocess('audio-train', "900", "20")

#### Load spectrograms as matrices

In [14]:
X, y = load_features('audio-train', "20")
cleanup_split('audio-train')

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#### One-hot encode the inputs

In [16]:
import keras
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

l_enc = LabelEncoder()
l_enc.fit(y_train)
y_train_enc = l_enc.transform(y_train)
y_train_norm = np_utils.to_categorical(y_train_enc)

l_enc.fit(y_test)
y_test_enc = l_enc.transform(y_test)
y_test_norm = np_utils.to_categorical(y_test_enc)

Using TensorFlow backend.


#### Remove unnecessary variables from memory (To clear memory)

In [29]:
del X, y, y_train_enc, y_test_enc, y_train, y_test

NameError: name 'X' is not defined

#### Build the model

In [33]:
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(513, 800, 3)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(31))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

#### Train the neural network

In [32]:
epoch = 1
while epoch <= 20:
    model.fit(np.array(X_train), y_train_norm,
              batch_size=30,
              epochs=1,
              verbose=1, 
             validation_data=(np.array(X_test[:1000]), y_test_norm[:1000]))
    model.save_weights('spect_model_weights_' + str(epoch) + '.h5')
    epoch += 1

ValueError: could not broadcast input array from shape (513,800,3) into shape (513)

#### Save the model weights to a file

In [15]:
model.save_weights('spect_model_weights.h5')